In [2]:
from typing import Tuple,Dict,List

In [3]:
import numpy as np; import cvxpy;
from collections import OrderedDict
#crear una lista con los pi de mayor a menor, 
#
def knapsack_fract_greedy(a_weights: np.ndarray, a_values: np.ndarray, bound: int)-> np.ndarray:
    pi_array=a_values/a_weights
    indices_ordenados=np.argsort(pi_array)[::-1]
    capacidad_restante=bound
    mochila={}
    valor_en_mochila=0.
    for i in indices_ordenados:
    #while carga<bound and contador<num_elementos: #esta ultima condición nos ayuda a no salirnos del index del array
        if a_weights[i]<=capacidad_restante:#entra_entero:
            mochila[i]=a_weights[i]
            capacidad_restante -= a_weights[i]
            valor_en_mochila += a_values[i]
        elif capacidad_restante > 0.:
            mochila[i]=capacidad_restante
            capacidad_restante=0.
            valor_en_mochila += a_values[i]*mochila[i]/a_weights[i]
        else: 
            mochila[i]=0
    
    mochila=dict(OrderedDict(sorted(mochila.items())))
    return valor_en_mochila,mochila
        
pesos=np.array([1,4,2,3])
valores=np.array([4,4,2,2])
print(knapsack_fract_greedy(pesos,valores,4))

ModuleNotFoundError: No module named 'cvxpy'

In [ ]:
import cvxpy as cp ; #import psycipopt
def knapsack_fract_lin_prog(a_weights: np.ndarray, a_values: np.ndarray, bound: float)-> tuple[float, dict]:
    c=a_values #supongo que x es un vector que me dirá cuánto peso cojo para llenar la muchila con el mayor valor
    pi_array=a_values/a_weights
    x = cp.Variable(len(a_weights))
    l_constraint=[x<=a_weights,cp.sum(x)<=bound]
    cost =cp.sum(pi_array @ x)    #c.T @ x
    objetive = cp.Maximize(cost)
    problema= cp.Problem(objetive,l_constraint)
    problema.solve()
    return round(problema.value,4), np.round(x.value,4)

pesos=np.array([1,2,2,4])
valores=np.array([4,4,2,2])
print(knapsack_fract_lin_prog(pesos,valores,5))




(10.0, array([ 1.,  2.,  2., -0.]))


In [40]:
def knapsack_01_pd_matrix(a_weights: np.ndarray, a_values: np.ndarray, bound: int)-> np.ndarray:
    ordenado = sorted(list(zip(a_weights,a_values)))
    
    matriz=np.zeros((len(ordenado),bound+1))
    for fila in range(len(ordenado)):
        for columna in range(bound+1):
            if columna == 0:
                matriz[fila, columna] = 0
            elif fila-1>=0:           #cogemos el [0] porque las tuplas que hay en ordenado tienen como primer elemento son los pesos
                if columna-ordenado[fila][0]>=0: #entramos aqui cuando podamos coger el elemento que este en la fila anterior,en la columna w-wi
                    matriz[fila,columna]=max(ordenado[fila][1]+matriz[fila-1,columna-ordenado[fila][0]], matriz[fila-1,columna])           
                else:
                    matriz[fila,columna]=matriz[fila-1,columna]
            elif columna>=ordenado[fila][0]:  #llegamos aquí cuando estamos en la 1a fila 
                matriz[fila,columna]=ordenado[fila][1] #si el elemento entra,entramos dentro
                
            else: # no entra el elemento
                matriz[fila, columna] = 0
                
    return matriz


pesos=np.array([4, 4, 5])
valores=np.array([10, 11, 15])
pesos2=np.array([2,3,4,5])
valores2=np.array([3,4,5,6])
m = knapsack_01_pd_matrix(pesos, valores, 8)
print(m)
print(sorted(list(zip(pesos,valores))))
        

[[ 0.  0.  0.  0. 10. 10. 10. 10. 10.]
 [ 0.  0.  0.  0. 11. 11. 11. 11. 21.]
 [ 0.  0.  0.  0. 11. 15. 15. 15. 21.]]
[(4, 10), (4, 11), (5, 15)]


In [41]:
        

def knapsack_01_int_prog(a_weights: np.ndarray, a_values: np.ndarray, bound: int)-> Tuple[int, Dict]:
    ordenado = sorted(list(zip(a_weights,a_values)))
    matriz=knapsack_01_pd_matrix(a_weights,a_values,bound)
    print(matriz)
    num_cols=bound+1
    i=len(a_weights)-1 ; j=bound ;d={}
    last=matriz[i][j] #el valor total que nos llevamos con la solución óptima al problema de la mochila 01
    
    while i>=0:    
        if i==0: #si estamos en la fila con el 1er elemento (fila 0), sabemos que su casilla superior sería cero
            upper=0
        else:
            upper=matriz[i-1][j]
        if last == upper:  #no cogemos ese elemento en la solución óptima
                        
            d[i+1]=0
            i -= 1 ; last=matriz[i][j] #nos quedamos con el elemento de arriba
        
        else:
            
            d[i+1]=1
            i -= 1 ; j -= ordenado[i][0] # w-w_i
            last=matriz[i][j]
        print(last)
            
    return d   

m=knapsack_01_int_prog(pesos2,valores2,5)   
print(m)    
         

[[0. 0. 3. 3. 3. 3.]
 [0. 0. 3. 4. 4. 7.]
 [0. 0. 3. 4. 5. 7.]
 [0. 0. 3. 4. 5. 7.]]
7.0
7.0
3.0
5.0
{4: 0, 3: 0, 2: 1, 1: 1}


In [29]:
#db["ejs mongo"].aggregate([{$group:{_id:"$code",count:{$count: {}}},$sort:{count:-1}}, {limit:10}]);
#db["ejs mongo"].aggregate([{$match:{code:"ESP"}},{$group:}])

2

In [13]:
pesos=np.array([7,2,2,4])
valores=np.array([4,8,2,2])
lista_de_tuplas=sorted(list(zip(pesos,valores)))
indices_ordenados=np.argsort(np.array(zip(pesos,valores)))[::-1]

print(lista_de_tuplas)
print(indices_ordenados)

[(2, 2), (2, 8), (4, 2), (7, 4)]
[0]


In [10]:
for i in range(3,0,-1):
    print(i)

3
2
1
